In [7]:
%%capture
%load_ext autoreload
%autoreload 2
%cd /home/ubuntu/projects/hyper-sloth

In [8]:
from speedy_utils.all import *
from llm_utils import *
from fastcore.all import *



#### Create a LLM model

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Choose any GPU you want to use
from vllm.lora.request import LoRARequest
from vllm import LLM, SamplingParams
import torch
# setup logger
import logging
from os import pipe

if "llm" in locals():
    del llm  #
    torch.cuda.empty_cache()
    
llm = LLM(
    model="outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH-merge",
    tensor_parallel_size=1,
    task="generate",
    enforce_eager=True,
    dtype=torch.bfloat16,
    max_model_len=16384,
    enable_lora=True,
    # quantization="bitsandbytes", load_format="bitsandbytes",gpu_memory_utilization=0.95
)

tokenizer = llm.get_tokenizer()

from datasets import load_dataset

# Load the GSM8K dataset
gsm8k = load_dataset("gsm8k", "main")
test = gsm8k["test"]


In [ ]:
# Prepare prompts for GSM8K evaluation
all_questions = [item["question"] for item in test][:100]
standardized_prompts = [
    tokenizer.apply_chat_template(
        [{"role": "user", "content": f"{question}\nSolve step by step and put your final numerical answer inside \\boxed{{}}"}],
        tokenize=False,
        add_generation_prompt=True,
    )
    for question in all_questions
]

# Set sampling parameters for deterministic generation
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, top_k=64, max_tokens=10000)

# Generate responses for all questions
outputs = llm.generate(standardized_prompts, sampling_params)
all_outputs = [output.outputs[0].text for output in outputs]


In [12]:
def get_final_output(response):
    try:
        return int(response.split("\\boxed{")[1].split("}")[0])
    except:
        return None

In [13]:
final_outputs = [get_final_output(response) for response in all_outputs]
accs = []
num_error = 0
for i, gt in enumerate(test):
    pred = final_outputs[i]
    try:
        num = gt['answer'].split('####')[1]
        # num = all_answers.append(int(num))
        # all_answers.append(num)
        num = int(num)
        pred = int(pred)
        accs.append(num == pred)
    except:
        num_error += 1
        accs.append(0)
        pass
        # all_answers.append(None)


In [15]:
np.mean(accs), num_error/len(final_outputs)

(0.5473843821076573, 0.17816527672479152)

In [16]:
from IPython.display import Markdown
Markdown(all_outputs[12].replace('<think>', '\n---\n').replace('</think>', '\n---\n'))


---

Lemon tree: $90, 7 lemons/year, $1.5 each, $3/year water/food.

Costs: planting, water/food, lemons.

Total cost: $90 + (7 * $3) + (7 * $1.5).

Calculate each cost.

Planting: $90.

Water/food: 7 * $3 = $21.

Lemons: 7 * $1.5 = $10.5.

Total cost: $90 + $21 + $10.5 = $121.5.

Earnings: 7 lemons/year * $1.5 = $10.5/year.

Earnings: $10.5/year.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5.

Earnings vs. cost: $10.5 - $90 = -$79.5